In [3]:
import spacy

In [4]:
import pandas as pd
from spacy import displacy
from spacy.matcher import Matcher

In [5]:
import en_core_web_sm


In [6]:
nlp = en_core_web_sm.load()

In [7]:
ner=nlp.get_pipe("ner")

In [8]:
import json

In [9]:
with open('training_data.json') as fp:
    training_data = json.load(fp)
with open('training_data_new.json') as fp:
    training_data2 = json.load(fp)   

In [10]:
for i in range(len(training_data2["annotations"])):
    a = training_data2["annotations"][i]
    training_data2["annotations"].append(a)
len(training_data["annotations"])

30

In [11]:
res = []
for i in training_data["annotations"]:
    if i not in res:
        res.append(i)

In [12]:
for text, annotations in res:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

In [13]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [14]:
import random
from spacy.util import minibatch, compounding
from pathlib import Path

In [15]:
a=0
b=0
c=0
loss=[]
all_loss=[]
bat=[]
i=0
with nlp.disable_pipes(*unaffected_pipes):
    for iteration in range(30):
        random.shuffle(res)
        losses = {}
        batches = minibatch(res, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            i=i+1
            #print("Losses", losses)
            all_loss.append(losses)
            if losses['ner'] < 1:
                #print("Losses", losses)
                loss.append(losses)
                bat.append(batch)
                #print(len(loss))
                a=b
                b=i
                c=b-a
                print("difference between this value and the previous one: ",c)
                print("i: ",i)

C:\Users\user\anaconda3\lib\site-packages\spacy\language.py:479: UserWarning: [W030] Some entities could not be aligned in the text "Don't buy this until you've looked at all of the e..." with entities "[[95, 105, 'PRODUCT']]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)


difference between this value and the previous one:  145
i:  145
difference between this value and the previous one:  8
i:  153
difference between this value and the previous one:  40
i:  193
difference between this value and the previous one:  8
i:  201
difference between this value and the previous one:  16
i:  217


In [16]:
text=[]
text.append("I love how easy it is to disinfectant my counter without getting a disinfectant spray out using a cloth that may or may not be clean. Clorox wipes are so easy to use!! This review was collected as part of a promotion.")
text.append("I didn't know what to expect when I first tried these products but as soon as I started to pour out the conditioner I couldn't believe how thick and rich it was. I knew I was on to something good. The shampoo left my hair feeling so soft and easy to manage. The conditioner was a surprise too. Not the type that leaves you hair so soft that it is unmanageable. My scalp never felt so good after using these products. No itch here! The scent is not overpowering, just right! This review was collected as part of a promotion.")
text.append("These Clorox wipes are an excellent product. I can use them with confidence and know that what I am cleaning will be thoroughly cleaned and sanitized when I use them. They are convenient to use and great quality. This review was collected as part of a promotion.")
text.append("I use the disinfecting wipes in the bathroom mostly. They are great for cleaning and disinfecting the vanity top, getting up little smudges on the floor, cleaning the cup holder, etc. There are so many things I use it for and it does a great job on all of them. An added bonus to these is the smell. Not only do they work great, they smell good too.")
text.append("I love to vaccum my home every sunday using your perfect product")
text.append("These DVD's are so rad and fun")
text.append("This DVD set arrived in a timely manner, and so far I am enjoying this series. I look forward to watching more.")
text.append("love these wipes. Make cleaning the bathroom a breeze. Kitchen, too. Last a long time and are durable This review was collected as part of a promotion.")
text.append("This is the first time I used the cream, I usually use the liquid. It was awesome.")
text.append("Chester's Cheese Puffcorn is my go-to late night snack every night. I usually eat 2 - 2 1/2 bags a week. It is delicious and low in price. I tell all my friends about it.")
text.append("These little airy corn puffs are addictive!! I love them!")
text.append("I remember my mother using this cleaner as a kid. I needed to make sure the bathroom was free of any issues, seeing how we had some sickness this year. I was looking online for this cleaner, so glad I found a bottle. Now, just make it easier to buy. I only had one place in town that carried this on the shelf.")

In [17]:
for i in range(len(text)):
    doc=nlp(text[i])
    #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    displacy.render(doc,style="ent")

C:\Users\user\anaconda3\lib\site-packages\spacy\displacy\__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
